[View in Colaboratory](https://colab.research.google.com/github/fatmas1982/fh2017/blob/master/new_york_times_articles.ipynb)

In [4]:
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from collections import Counter
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 
import numpy as np
import scipy.stats.stats as st
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#from stemming.porter2 import stem
from nltk import PorterStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from string import digits
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv

import re
from nltk import word_tokenize
import string
!pip install gensim
import  gensim.models as md
from gensim.models.phrases import Phrases, Phraser


    100% |████████████████████████████████| 22.6MB 1.9MB/s 
    100% |████████████████████████████████| 1.4MB 9.4MB/s 
    100% |████████████████████████████████| 133kB 23.3MB/s 
    100% |████████████████████████████████| 61kB 17.1MB/s 
    100% |████████████████████████████████| 4.2MB 10.4MB/s 
    100% |████████████████████████████████| 552kB 20.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /content/.cache/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:


from google.colab import files
uploaded = files.upload()


for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


KeyboardInterrupt: ignored

In [8]:
import urllib.request

print('Beginning file download with urllib2...')

url = 'https://www.kaggle.com/nzalake52/new-york-times-articles/downloads/nytimes_news_articles.txt/1'
urllib.request.urlretrieve(url, '/nytimes_news_articles.txt')

Beginning file download with urllib2...


('/nytimes_news_articles.txt', <http.client.HTTPMessage at 0x7fa1ef505e10>)

In [0]:
path_database='/home/fsg/PH/NYT_output/' 

pragraph_index='pragraph_index.csv'
Sentences='Sentences.csv'
Sentences_not_stops='Sentences_not_stops.csv'

In [10]:
!ls datalab


adc.json


In [0]:
!ls

In [0]:
'''
Write Excell sheet
'''
def save_file_to_database(data_rows,path_database,file_databbase,header_list):#header_list=['index','text']
    outfile = open(path_database+file_databbase,'w')
    writer=csv.writer(outfile)
    #header_list=['uuid','paragraph','doc_id']
    i=0
    for line in data_rows:
        row=[i,line]#,'paragraph no.'+str(i)]
        if i==0:
            
            writer.writerow(header_list)
            writer.writerow(row)
        else:
            #print('ff')
            writer.writerow(row)
        i+= 1
        #outfile.close()
            

In [0]:
def read_text_from_database(path_database,file_databbase):
    queue_paragraph=[]
    #f = open(sys.argv[1], 'rt')
    outfile = open(path_database+file_databbase,'rt')
    try:
                
        reader=csv.reader(outfile)
        for row in reader:
            queue_paragraph.append(row)
            #print (row)
    finally:
        print ("row")
        outfile.close()
        
    return queue_paragraph

In [0]:
def read_cvs_by_pands(path_database,file_databbase,index_col, header):
    return pd.read_csv(path_database+file_databbase,index_col=index_col,header=header)

In [0]:
def pragraph_to_setnences(str):
    return sent_tokenize(str)

In [0]:
def stopwords_list():
    stopwordsFile = open('./input/stopwords/stopwords.txt')
    stopwordsFile.seek(0)
    stopwordsV1 = stopwordsFile.readlines()
    stopwordsV2 = []
    for sent in stopwordsV1:
        sent.replace('\n', '')
        new_word = sent[0:len(sent) - 1]
        stopwordsV2.append(new_word.lower())
    return stopwordsV2

In [0]:
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['--','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
numbers=[1,2,3,4,5,6,7,8,9]
stopwordsV2=stopwords_list()
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
def remove_stopword_sentences(str):
   
            
    list_word=[]
    tokenizer = RegexpTokenizer("[\w']+")
    
    words=tokenizer.tokenize(str)
    for word in words:
        new_word = word.encode('ascii', 'ignore').decode('utf-8')
        if new_word != '':
    
            english_stops = set(stopwords.words('english'))
           
            list_word=[new_word for new_word in words if new_word.lower() not in english_stops
                       and new_word.lower() not in new_stop_words 
                       and new_word.lower() not in new_stop_words2 
                       and  not new_word.lower().isdigit() 
                       and new_word.lower() not in digits 
                       and new_word.lower() not in  numbers and word.lower() not in stopwordsV2
                       and new_word.lower() not in string.punctuation]
    
  
    
    return list_word#(stem(setem_word for setem_word in  ([word for word in words if word not in english_stops and word not in new_stop_words])))

# Courps to CSV docs

In [0]:
import pandas as pd
def courps_to_CSV_docs():
    #Reading the news articles file
    nyTimesFile = open('./input/new-york-times-articles/nytimes_news_articles.txt', encoding='latin-1')
    nyTimesFile.seek(0)
    nyTimesV1 = nyTimesFile.readlines()
    nyTimesTemp = []
    nyTimesURL = []

    for i in range(0, len(nyTimesV1)-1):
        if re.findall('URL', nyTimesV1[i]) == []:
            sent = sent + nyTimesV1[i]
            if (re.findall('URL', nyTimesV1[i+1]) != []) and (i+1 < len(nyTimesV1)):
                nyTimesTemp.append(sent.strip())
        else:
            sent = ''
            nyTimesURL.append(nyTimesV1[i])

    for i in range(0, len(nyTimesTemp)):
        nyTimesTemp[i] = nyTimesTemp[i]+'articleID'+str(i)
    print(len(nyTimesTemp))
    header_list=['index','text']
    save_file_to_database(nyTimesTemp,path_database,pragraph_index,header_list)
    '''for i in range(1):
        print(i,"============================================")
        print("============================================")'''
    #nytimes = preProcessor(nyTimesTemp)
    print("============================================")
    #print(nytimes)

In [0]:
paragraphs=read_cvs_by_pands(path_database,pragraph_index,None,0)
paragraphs

# Doc to sentence

In [0]:
def paragraphs_to_sentece(pragraphs):
    sentenses_list=[]
    
    for index_p in  range(len(pragraphs)):
        #print(index_p)
        #print("pppppppppppppppppppp")
        #print(pragraphs[index_p])
        
        setnences=pragraph_to_setnences(pragraphs[index_p])
        print("sssssssssssssssssssssssssss")
        #print(setnences)

        for indexs in range(len(setnences)):
            row=[]
            #print(setnences)
            row.append(index_p)
            row.append(indexs)
            row.append(setnences[indexs])
            sentenses_list.append(row)
    header_list=['index_P','index_sent','sentence']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

#paragraphs_to_sentece(paragraphs.text)

In [0]:
def  paragraphs_to_sentece_Not_stop_word(pragraphs):
    #words_list=[]
    sentenses_list=[]
    
    for index_p in  range(len(pragraphs)):
        setnences=pragraph_to_setnences(pragraphs[index_p])
        
        for indexs in range(len(setnences)):    
            #print("Sentence No. ",indexs,": ",setnences[indexs],"\n")
            words=remove_stopword_sentences(setnences[indexs])
            wordsent=''
            row=[]
            
            row.append(index_p)
            row.append(indexs)
            row.append(words)
            sentenses_list.append(row)
    header_list=['index_P','index_sent','words_not_stop']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences_not_stops, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

#paragraphs_to_sentece_Not_stop_word(paragraphs.text)

# Sentence To LESK

In [0]:
'''
this function for compute lesk of word in sentence
'''

def lesk_word_sentence(sentence,word):
    from nltk.wsd import lesk
    lesk_synset=''
    #lesks= []
    #for word in words:
    #disambiguated=lesk(context_sentence=sentence, ambiguous_word=word)
    disambiguated=lesk(sentence,word, 'n')
    #print(disambiguated)
    #if disambiguated is not None:
    lesk_synset=disambiguated
    #else:
    #lesk_synset=0
    #print("Word is: ",word,"\n LESK: ",lesk(sentence,word, 'n'),"\n Sentence: ",sentence )
        
    return lesk_synset

#lesk("Computer science is a discipline that spans theory and practice","science")

#sent = 'people should be able to marry a person of their choice'.split()
#lesk(sent, 'able')

In [0]:
lesk_word_sentence("I love you","love")

Synset('sexual_love.n.02')

In [0]:
def sent_lesks():
    df_Sentences=read_cvs_by_pands(path_database,Sentences,None,0)
    df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
    sentences=df_Sentences.sentence
    words_not_stop=df_Sentences_not_stops.words_not_stop#[0]#['words_not_stop'][0]
    lesk_df_list=[]
    file=open('./NYT_output/file.txt', 'a+')
    print(len(df_Sentences))
    for i in range(317989):#len(df_Sentences)):
        lesks=[]
        lesks_name=[]
        row=[]
        if words_not_stop[i]!='[]':

            words_not_stop[i]=words_not_stop[i].replace("'",'').replace("[",'').replace("]",'')
            sentences[i]=sentences[i].replace("'",'').replace("[",'').replace("]",'')       
            words_not_stop_list=words_not_stop[i].split(',')
            #print(words_not_stop_list)
            #print(sentences[i])
            for word in words_not_stop_list:
                #print(word)
                l=lesk_word_sentence(sentences[i],word)
                #print("l",l)
                if l is not None:
                    lesks.append(l)
                    lesks_name.append(l.name())
        #print("888888888888888888888888888888888888888888")
        row.append(df_Sentences_not_stops.index_P[i])
        row.append(df_Sentences_not_stops.index_sent[i])
        row.append(lesks)
        row.append(lesks_name)
        lesk_df_list.append(row)
        print(i)
        file.write(str(i))

    #print(lesk_df_list)
    header_list=['index_P','index_sent','lesks','lesks_name']
    df = pd.DataFrame(lesk_df_list, columns=header_list)
    
    df.to_csv(path_database+"lesks.csv", encoding='utf-8', index=False)  
    #return df




sent_lesks()

317989
0


/home/fsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/fsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
#https://www.kaggle.com/nzalake52/text-search-text-mining/notebook
#We will be implementing a simple search engine using nltk in python
#With the help of TF-IDF ranking and cosine similarity we can rank the documents and get the desired output.
#Following is the code for the same, many machine learning algorithms can be applied and the use of this search engine 
#can be extended.
#This is the simplest search engine implementation




#Creating the function for preprocessing the text file, the functions does the following:
#Removing the URLs in the file
#Removing blank lines
#Converting the letters that were not read properly due to encoding, to be viewed properly
#Removing stopwords from the text
#Removing the punctuations form the text

def preProcessor(textFile):
    print('Starting pre-processing of the corpus..')
    print('Start: Word Tokenizing')

    textFilev1 = []
    textFilev1 = [word_tokenize(sent) for sent in textFile]
    
    
    print('Stop: Word Tokenizing')
    print('Start: ASCII encoding for special characters')

    textFilev2 = []
    for sent in textFilev1:
        new_sent = []
        for word in sent:
            new_word = word.encode('ascii', 'ignore').decode('utf-8')
            if new_word != '':
                new_sent.append(new_word)
        textFilev2.append(new_sent)

    print('Stop: ASCII encoding for special characters')
    print('Start: Stopwords Removal')

    stopwordsFile = open('./input/stopwords/stopwords.txt')
    stopwordsFile.seek(0)
    stopwordsV1 = stopwordsFile.readlines()
    stopwordsV2 = []
    for sent in stopwordsV1:
        sent.replace('\n', '')
        new_word = sent[0:len(sent) - 1]
        stopwordsV2.append(new_word.lower())

    textFilev1 = []
    for sent in textFilev2:
        new_sent = []
        for word in sent:
            if word.lower() not in stopwordsV2:
                new_sent.append(word.lower())
        textFilev1.append(new_sent)

    print('Stop: Stopwords Removal')
    print('Start: Punctuation Removal')

    textFilev2 = []
    for sent in textFilev1:
        new_sent = []
        for word in sent:
            if word not in string.punctuation:
                new_sent.append(word)
        textFilev2.append(new_sent)

    print('Stop: Punctuation Removal')
    print('Start: Phrase Detection')

    textFilev1 = []
    common_terms = ["of", "with", "without", "and", "or", "the", "a", "so", "and"]
    phraseTrainer = Phrases(textFilev2, delimiter=b' ', common_terms=common_terms)
    phraser = Phraser(phraseTrainer)
    for article in textFilev2:
        textFilev1.append((phraser[article]))

    print('Stop: Phrase Detection')

    return textFilev1

In [0]:
#Function for creating intermediate index
def file_indexing(file):
    fileIndex = {}
    for index, word in enumerate(file):
        if word in fileIndex.keys():
            fileIndex[word].append(index)
        else:
            fileIndex[word] = [index]
    return fileIndex

#building final index
def fullIndex(intIndex):
    totalindex = {}
    for fileName in intIndex.keys():
        for word in intIndex[fileName].keys():
            if word in totalindex.keys():
                if fileName in totalindex[word].keys():
                    totalindex[word][fileName].extend(intIndex[fileName][word][:])
                else:
                    totalindex[word][fileName] = intIndex[fileName][word]
            else:
                totalindex[word] = {fileName : intIndex[fileName][word]}
    return totalindex

In [0]:
nyTimesIndex = {}
for sent in nytimes:
    nyTimesIndex[' '.join(sent)] = file_indexing(sent)

nyTimesIndexV1 = fullIndex(nyTimesIndex)

#Functions to create one word or phrase query search
def wordSearch(word):
    if word in nyTimesIndexV1.keys():
        return [file for file in nyTimesIndexV1[word].keys()]


def phraseQuery(string):
    lists, result = [], []
    for word in string.split():
        lists.append(wordSearch(word))
    setList = set(lists[0]).intersection(*lists)
    for fileName in setList:
        result.append(fileName)
    return result



In [0]:
searchResult = phraseQuery('white  house')
searchResult1 = []
for file in wordSearch('white'):
    searchResult1.append(file)
for file in wordSearch('house'):
    if file not in searchResult1:
        searchResult1.append(file)


In [0]:
#Making use of TF-IDF ranking to rank the documents given by the searches
#Also using similarity metrics (Cosine similarity) to get the similarity scores between both the documents 
#from both search results
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer().fit(searchResult1)
searchResult1TFIDF = tfidf.transform(searchResult1)
searchResultTFIDF = tfidf.transform(searchResult)
sims = cosine_similarity(searchResult1TFIDF, searchResultTFIDF)


In [0]:
#Now putting the cosine similarity results into a data frame
#Sorting the dataframe by score and getting the most similar and appropriate 30 documents from the search results
cosineSum = []
for ind in range(len(sims)):
    cosineSum.append(sum(sims[ind]))

sumDF = pd.DataFrame({'score':cosineSum})
sumDF['index'] = [i for i in range(len(cosineSum))]
sumDF.sort_values(by='score', inplace=True, ascending=False)

for ind in sumDF['index']:
    print(nyTimesURL[int(searchResult1[ind][str(searchResult1[ind]).find('articleid')+9:])], '\n')


In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))